In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

# for hierarchical clusterization
from scipy.cluster.hierarchy import dendrogram, linkage  
from scipy.spatial.distance import  pdist
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# system
from datetime import datetime
import os
sns.set()

In [256]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.model_selection import GridSearchCV

In [258]:
data = pd.read_csv("data//raai_school_2024.csv", sep = ";")
data["payment_period"] = pd.to_datetime(data["payment_period"])
data['user_id'] = data['house_tkn'].astype(str) + '_' + data['flat_tkn'].astype(str)
data.head()

,house_tkn,flat_tkn,payment_period,income,debt,raised,volume_cold,volume_hot,volume_electr,user_id
0,2,23170,2023-01-01,2073.48,2046.36,0.00,5.33,6.91416,199.50,2_23170
1,2,23170,2023-03-01,1525.44,2484.21,2404.08,5.19,4.22532,186.00,2_23170
2,2,23170,2023-04-01,1762.49,2337.36,1672.29,5.00,4.82284,227.50,2_23170
3,2,23170,2023-05-01,1868.41,0.00,4099.85,5.08,5.13227,242.03,2_23170
4,2,23170,2023-07-01,2682.54,1933.29,0.00,8.64,6.91416,362.00,2_23170


In [259]:
data.shape

(3206079, 10)

## Descriptional analysis

На что надо обратить внимание

1. Как можно заметить внизу в данных есть заметные выбросы. Но они легко отбрасываются при помощи 5% перцентиля.
2. В данных $volume$ есть пробелы заполенные NaN. Их не получится просто отбросить учитывая что они занимают почти половину строк. Факт: было подтверждено что у многих вместо горячей воды стоят какие-то собственные приспособление так что за горячую воду они не платят. Осталось догадаться, понять есть ли люди без элекричества или дом просто пустует.  

In [ ]:
data.describe()

In [ ]:
features = ['income', 'debt', 'raised']
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(8, 6))

for i, feature in enumerate(features):
    lower_bound = np.percentile(data[feature].dropna(), 5)
    upper_bound = np.percentile(data[feature].dropna(), 95)
    filtered_data = data[feature][(data[feature] >= lower_bound) & (data[feature] <= upper_bound)].dropna()
    axes[i].hist(filtered_data, bins=20, alpha=0.5)  # Apply log transformation
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

### Searching anomalies with scatter plots and basic deduction

In [ ]:
sns.scatterplot(x = 'volume_cold', y = 'volume_hot', data = data)
plt.title('Points of cold, hot volume')

In [ ]:
print(f"Meanwhile median for cold water: {data.volume_cold.median()}, hot water: {data.volume_hot.median()}")

### Мини исследование отсутствующих значений.

In [ ]:
data['volume_cold_na'] = data.volume_cold.isna()
data['volume_hot_na'] = data.volume_hot.isna()
data['volume_electr_na'] = data.volume_electr.isna()

counts = pd.DataFrame({
    'volume_cold_na': [data['volume_cold_na'].sum(), 
                       data[['volume_cold_na', 'volume_hot_na']].all(axis=1).sum(),
                       data[
                           ['volume_cold_na', 'volume_electr_na']].all(axis=1).sum()],
    'volume_hot_na': [data[
                           ['volume_cold_na', 'volume_hot_na']].all(axis=1).sum(),
                       data['volume_hot_na'].sum(),
                       data[['volume_hot_na', 'volume_electr_na']].all(axis=1).sum()],
    'volume_electr_na': [data[['volume_cold_na', 'volume_electr_na']].all(axis=1).sum(),
                       data[['volume_hot_na', 'volume_electr_na']].all(axis=1).sum(),
                       data['volume_electr_na'].sum()]
}, index=['volume_cold_na', 'volume_hot_na', 'volume_electr_na'])

plt.figure(figsize=(8, 6))
sns.heatmap(counts, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel('Missing Values')
plt.ylabel('Missing Values')
plt.title('Intersection of Missing Values in School Data')

plt.show()

In [ ]:
cold_no_hot_water = data['volume_hot_na'] & (1 - data['volume_cold_na'])
cold_and_hot_water = (1 - data['volume_hot_na']) & (1 - data['volume_cold_na'])
no_cold_no_hot_water = data['volume_hot_na'] & data['volume_cold_na']

features = ['income', 'raised', 'volume_cold', 'volume_electr']

fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(8, 20))

for i, feature in enumerate(features):
    lower_bound = np.percentile(data[feature].dropna(), 5)
    upper_bound = np.percentile(data[feature].dropna(), 95)
    filtered_indices = (data[feature] >= lower_bound) & (data[feature] <= upper_bound)

    filtered_cold_no_hot_water = data[feature][cold_no_hot_water & filtered_indices].dropna()
    filtered_cold_and_hot_water = data[feature][cold_and_hot_water & filtered_indices].dropna()
    filtered_no_cold_no_hot_water = data[feature][no_cold_no_hot_water & filtered_indices].dropna()

    axes[i].hist(np.log(filtered_cold_no_hot_water + 1), alpha=0.5, label='Cold Only', bins=20)
    axes[i].hist(np.log(filtered_cold_and_hot_water + 1), alpha=0.5, label='Cold & Hot', bins=20)
    axes[i].hist(np.log(filtered_no_cold_no_hot_water + 1), alpha=0.5, label='No Cold & No Hot', bins=20)
    
    axes[i].set_xlabel("Log " + feature)
    axes[i].set_ylabel('Frequency')
    axes[i].legend()

plt.title("Difference between flats with or without hot water")
plt.tight_layout()
plt.show()

### Исследование по электричеству

Выводы:
- если электричества нет то совершенно не факт что нет показателей воды.
- в 37% случаев есть показания воды хотя нет электрчества
- если есть показания горячей воды то почти наверное есть холодная вода, хотя один процент несколько подозрителен. Были замечены некоторые дома с подозрительным количеством жителей у которых только горячая вода, хотя большая часть все же распределена по небольшим группам
- внизу я еще пыталась понять как выглядят те пользователи без холодной воды и стало понятно что большая часть - индивидуальные случаи максимум с парой показаний, вероятно у них просто что-то сломалось. С другой стороны есть дома с массовым отсутствием воды (типа 50 жителей без холодной воды судя по всему по 9 месяцев). Странный дом, а именно 144191 имеет аж 124 жителя которые живут без холодной воды до 7 месяцев

In [ ]:
print("Percent of houses with hot & cold water:", sum((1 - data['volume_hot_na']) & (1 - data['volume_cold_na'])) / data.shape[0] * 100)
print("Percent of houses with only cold water:", sum(data['volume_hot_na'] & (1 - data['volume_cold_na'])) / data.shape[0] * 100)
print("Percent of houses with only hot water:", sum(data['volume_cold_na'] & (1 - data['volume_hot_na'])) / data.shape[0] * 100)
print("Percent of houses with no hot & no cold water:", sum((1 - data['volume_hot_na']) & (1 - data['volume_cold_na'])) / data.shape[0] * 100)

print("-----------------------------------------------------------------------------------------------------")

print("Percent of houses with no electricity yet cold water:", sum(data['volume_electr_na'] & (1 - data['volume_cold_na'])) / data.shape[0] * 100)
print("Percent of houses with no electricity yet hot water:", sum(data['volume_electr_na'] & (1 - data['volume_hot_na'])) / data.shape[0] * 100)
print("Percent of houses with no electricity yet cold & hot water:", sum(data['volume_electr_na'] & (1 - data['volume_cold_na']) & (1 - data['volume_hot_na'])) / data.shape[0] * 100)

### Выделение пользователей у которых нет горячей воды / отключение выборочное

In [ ]:
data['month_of_admission'] = data['payment_period'].map(lambda dt: dt.strftime('%Y-%m'))
data.groupby('month_of_admission')["volume_hot_na"].sum().to_frame("count").reset_index().plot(kind='bar', x='month_of_admission', y='count', title = "Distribution of no-hot-water entries between months")

In [ ]:
print(f"Average number months of payment: {grouped.size().mean()}")

In [ ]:
data = data.sort_values('payment_period')
grouped = data.groupby('user_id')

def check_missing_months(user_data):
    # Sort by 'month_of_admission'
    user_data = user_data.sort_values('payment_period')
    # user_data['payment_period'] = pd.to_datetime(user_data['payment_period'])
    
    first_month = user_data['payment_period'].min()
    last_month = user_data['payment_period'].max()
    expected_months = pd.date_range(first_month, last_month, freq='MS', inclusive="both")

    # Find missing months
    missing_months = expected_months[~expected_months.isin(user_data['payment_period'])]
    exclude_months = pd.to_datetime(["2023-10-01", "2023-12-01"])
    missing_months = missing_months[~missing_months.isin(exclude_months)]
    return missing_months

number_skips = 0
for user_id, user_data in grouped:
    i += 1
    missing_months = check_missing_months(user_data)
    if missing_months is not None and not missing_months.empty:
        if number_skips < 100:
            print(f"User {user_id} has missing months: {missing_months.strftime('%Y-%m').tolist()}")
        number_skips += 1
print(number_skips)

In [ ]:
grouped[grouped["user_id"] == "100005_263170"]

In [ ]:
grouped = data.groupby('user_id')

def check_missing_months(user_data):
    # Sort by 'month_of_admission'
    user_data = user_data.sort_values('month_of_admission')
    user_data['month_of_admission'] = pd.to_datetime(user_data['month_of_admission'])
    
    first_month = user_data['month_of_admission'].min()
    last_month = user_data['month_of_admission'].max()
    expected_months = pd.date_range(first_month, last_month, freq='M')

    # Find missing months
    missing_months = expected_months[~expected_months.isin(user_data['month_of_admission'])]
    return missing_months

for user_id, user_data in grouped:
    missing_months = check_missing_months(user_data)
    if not missing_months.empty:
        print(f"User {user_id} has missing months: {missing_months.strftime('%Y-%m').tolist()}")

In [ ]:
# Plan how to devide users into groups with 

grouped = data.groupby('user_id')
users_with_no grouped["volume_hot_na"].sum().astype(int).reset_index(name="count")

### Отдельное исследование людей с горячей водой но не холодной :/

In [ ]:
weird_guys = data[data['volume_cold_na'] & (1 - data['volume_hot_na'])] # hot & no cold
weird_guys.shape

In [ ]:
print(f"Number of houses with citizens who got only hot water yet no cold {len(weird_guys.house_tkn.unique())}, average entries per house {weird_guys.shape[0] / len(weird_guys.house_tkn.unique())}\nMeanwhile there're average {data.shape[0] / len(data.house_tkn.unique())} entries per house in all data")

print(f"Number of unique citizens who got only hot water yet no cold {len(weird_guys.user_id.unique())}, average entries per user {weird_guys.shape[0] / len(weird_guys.user_id.unique())}\nMeanwhile there're average {data.shape[0] / len(data.user_id.unique())} entries per user in all data")

In [ ]:
plt.hist(np.log(weird_guys.house_tkn.value_counts()), bins = 10)
plt.title("Logged number entries with hot & no cold water per house")

plt.show()

In [ ]:
weird_guys.groupby("user_id").size().nlargest(10)

In [ ]:
weird_guys.groupby("house_tkn").size().nlargest(5)

In [ ]:
print("Cursed house:")
print("Number cursed flats in the cursed house:", weird_guys[weird_guys["house_tkn"] == 144191]["user_id"].nunique())
print("Number cursed entries for dwellers of the cursed house:", weird_guys[weird_guys["house_tkn"] == 144191]["user_id"].value_counts().mean())
print("Number super cursed dwellers of the house:", sum(weird_guys[weird_guys["house_tkn"] == 144191]["user_id"].value_counts() == 7))

In [ ]:
plt.hist(weird_guys.groupby("house_tkn")["user_id"].nunique(), bins = 20)
plt.title("Number users with hot & no cold water per house")
plt.show()

In [ ]:
weird_guys.groupby("house_tkn")["user_id"].value_counts()

In [ ]:
weird_guys.house_tkn.value_counts()

## Распределение по месяцам

In [ ]:
data['month_of_admission'] = data['payment_period'].map(lambda dt: dt.strftime('%Y-%m'))
data.groupby('month_of_admission').size().to_frame("count").reset_index().plot(kind='bar', x='month_of_admission', y='count')

## Preprocessing
- удаление двух месяцев у которых мало записей
- one-hot encoding месяцев
- выделение конкретных **домов** с аномально маленькими/большими значениями по каждому из столбцов
- EIF, возможно другие методы если он окажется неадекватным

-> Результатом является data_filtered

In [ ]:
data = data.reset_index(drop="True")

In [ ]:
data.columns

In [ ]:
# Deleting negative numbers from volumes
ensure_positive_features = ['income', 'raised', 'volume_cold', 'volume_hot', 'volume_electr']
for feature in ensure_positive_features:
    if sum(data[feature] < 0) != 0:
        print(f"Feature {feature} has unexpected negative values: {sum(data[feature] < 0)}")

### удаление двух месяцев у которых мало записей

In [ ]:
data_filtered = data[(data["month_of_admission"] != 10) & (data["month_of_admission"] != 12)]

### one-hot encoding месяцев

In [ ]:
# NATASHA's code here

### выделение конкретных **домов** с аномально маленькими/большими значениями по каждому из столбцов
- Наташа
- ОБЯЗАТЕЛЬНО сделай чистку месяцев до этого.
Удобно просто строить перцептили с pandas - посмотри как я выше это делала

- Идеи: чуваки которые не отправляют данные и не платят (у них капает за электроэнергию). Чуваки которые тратят много ровно по одному параметру. Чуваки которые тратят много по всем параметрам (а-ля многодетная семья) 

In [ ]:
# NATASHA's code here

### Artificial cutoff of the anomaly values

- for each volume take a percentile, but do not exclude NaN or 0

In [ ]:
numeric_features = ['income', 'debt', 'raised',
       'volume_cold', 'volume_hot', 'volume_electr']

for feature in numeric_features:
    threshold = data_numeric[feature].quantile(0.95)
    data_filtered[feature] = data_filtered[feature].clip(upper=threshold)

for feature in ['income', 'debt', 'raised']:
    threshold = data_numeric[feature].quantile(0.05)
    data_filtered[feature] = data_filtered[feature].clip(lower=threshold)

### Filling NaN values
1. Нахождение людей у которых просто нет горячей воды, заполнение таких значениями -1.
2. Отрицательные значения volume временно заполняем 0
3. Всех остальных имеет смысл заполнть медианой так как мы работаем на уровне записей

In [ ]:
data_filtered['month_of_admission'] = data_filtered['payment_period'].map(lambda dt: dt.month)
numeric_features = ['income', 'debt', 'raised','volume_cold', 'volume_hot', 'volume_electr']
data_numeric = data_filtered[numeric_features]

In [ ]:
# Filling all those who have no hot water with -1
user_ids_who_have_no_hot = data.groupby("user_id")["volume_hot_na"].agg(lambda x: x.any())
users_with_no_hot = data[data["user_id"].isin(user_ids_who_have_no_hot[user_ids_who_have_no_hot].index)]
users_with_no_hot["volume_hot"] = users_with_no_hot["volume_hot"].fillna(0)

data_numeric = pd.concat([users_with_no_hot, data_numeric[~data["user_id"].isin(users_with_no_hot["user_id"])]])
data_numeric = data_numeric[numeric_features]

In [ ]:
print(f"Entries with no hot water: {users_with_no_hot.shape[0]}")

In [ ]:
assert data_numeric.shape[0] == data_filtered.shape[0]

In [ ]:
# Filling negative values with 0 (hot water as I previouslt overlooked didn't have any so we don't wanna clear previous result)

data_numeric["volume_cold"] = data_numeric["volume_cold"].clip(lower=0)
data_numeric["volume_electr"] = data_numeric["volume_electr"].clip(lower=0)

In [ ]:
# Fill all the left NaNs with median
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')
scaler = StandardScaler()
data_numeric_scaled = scaler.fit_transform(imputer.fit_transform(data_numeric))

In [ ]:
# Resetting index (WARNING! that would make index inconsistent with original data)

data_numeric = data_numeric.reset_index(drop=True)
data_filtered = data_filtered.reset_index(drop=True)

### EIF for anomaly detection

In [ ]:
data.columns

In [ ]:
# Isolation forest from sklearn
from sklearn.ensemble import IsolationForest

n_trees_values = [1, 5, 10, 50]
max_samples_values = [0.33, 0.5, 0.7] 
contamination = 0.01

scores = []

subset_size = 0.1 
subset_indices = np.random.choice(data_numeric_scaled.shape[0], int(subset_size * data_numeric_scaled.shape[0]), replace=False)
data_subset = data_numeric_scaled[subset_indices]

for n_trees in n_trees_values:
    scores_ = []
    for max_samples in max_samples_values:
        print(f"Processing n_trees = {n_trees}, max_samples = {max_samples}")
        eif = IsolationForest(n_estimators=n_trees, max_samples=max_samples, contamination=contamination)  # Use max_samples
        eif.fit(data_subset)
    
        anomaly_scores = eif.decision_function(data_subset)
        
        score = np.mean(anomaly_scores)
        scores_.append(score)

    scores.append(scores_)

In [ ]:
plt.figure(figsize=(6, 4))
for i, n_trees in enumerate(n_trees_values):
    plt.plot(max_samples_values, scores[i], label=f'n_trees {n_trees}')
plt.xlabel('max_samples')
plt.ylabel('Anomaly score')
plt.title("Sklearn IsolationForest on 10% of data")
plt.legend()
plt.show()

In [ ]:
eif = IsolationForest(n_estimators=10, max_samples=0.7, contamination=0.01)
eif.fit(data_subset)

subset_size = 1000
subset_indices = np.random.choice(data_numeric_scaled.shape[0], subset_size, replace=False)
data_subset_2 = data_numeric_scaled[subset_indices]

original_subset = data_numeric.iloc[subset_indices].copy()
original_subset["anomaly"] = eif.predict(data_subset_2)

In [ ]:
features = ['income', 'debt', 'raised','volume_cold', 'volume_hot', 'volume_electr']

fig, axes = plt.subplots(len(features), len(features), figsize=(30, 28))

for i, feature1 in enumerate(features):
    for j, feature2 in enumerate(features):
        if i == j:
            axes[i, j].axis("off")
        else:
            axes[i, j].scatter(
                original_subset[feature1][original_subset["anomaly"] == 1],
                original_subset[feature2][original_subset["anomaly"] == 1],
                color="blue",
                label="Normal",
            )

            # Scatter plot for anomalies
            axes[i, j].scatter(
                original_subset[feature1][original_subset["anomaly"] == -1],
                original_subset[feature2][original_subset["anomaly"] == -1],
                color="red",
                label="Anomaly",
            )

            axes[i, j].set_xlabel(feature1)
            axes[i, j].set_ylabel(feature2)
            axes[i, j].legend()

plt.tight_layout()
plt.show()

In [ ]:
%%time
eif = IsolationForest(n_estimators=10, max_samples=0.7, contamination=0.01)
anomaly_predictions = eif.fit_predict(data_numeric_scaled)

In [ ]:
anomaly_indices = np.where(anomaly_predictions == -1)[0]

subset_size = 1000
subset_indices = np.random.choice(anomaly_indices, subset_size, replace=True)
original_subset = data_numeric.iloc[subset_indices].copy()
original_subset["anomaly"] = anomaly_predictions[subset_indices]

In [ ]:
normal_indices = np.where(anomaly_predictions == 1)[0]
data_filtered.iloc[normal_indices].describe()

In [ ]:
features = ['income', 'debt', 'raised','volume_cold', 'volume_hot', 'volume_electr']
fig, axes = plt.subplots(len(features), len(features), figsize=(30, 28))

for i, feature1 in enumerate(features):
    for j, feature2 in enumerate(features):
        if i == j:
            axes[i, j].axis("off")
        else:
            axes[i, j].scatter(
                original_subset[feature1][original_subset["anomaly"] == -1],
                original_subset[feature2][original_subset["anomaly"] == -1],
                color="red",
                label="Anomaly",
            )

            axes[i, j].set_xlabel(feature1)
            axes[i, j].set_ylabel(feature2)
            axes[i, j].legend()

plt.tight_layout()
plt.show()

In [ ]:
print(f"IF left {len(normal_indices)} normal, while {len(anomaly_indices)} are anomalies")

In [ ]:
# Saving normal data that was filtered
data_filtered.iloc[normal_indices].to_csv("filtered_unnormalized_data.csv", sep=";", index=False)

# Saving scaled data that has no NaN
replace_columns = ['income', 'debt', 'raised', 'volume_cold', 'volume_hot', 'volume_electr']
data_numeric_df = pd.DataFrame(data_numeric_scaled, columns=replace_columns)
data_filtered[replace_columns] = data_numeric_df
data_filtered.iloc[normal_indices].to_csv("filtered_scaled_data.csv", sep=";", index=False)

# Saving original data with anomaly tag

In [252]:
data_filtered["is_anomaly"] = 1

,house_tkn,flat_tkn,payment_period,income,debt,raised,volume_cold,volume_hot,volume_electr,month_of_admission,volume_cold_na,volume_hot_na,volume_electr_na,user_id
0,2,23170,2023-01-01,0.187621,-0.146264,-0.145109,0.470157,0.791438,-0.039733,1,False,False,False,2_23170
1,34737,208775,2023-01-01,-0.466695,-0.146041,-0.086698,-0.339071,0.322358,-0.039733,1,False,False,True,34737_208775
2,15899,163622,2023-01-01,-0.612532,-0.146041,-0.230077,-0.274897,-0.146721,-0.039733,1,False,False,True,15899_163622
3,94073,454760,2023-01-01,-0.798176,-0.146041,-0.485190,-0.210724,-0.615800,-0.039733,1,False,False,True,94073_454760
4,15899,163621,2023-01-01,-0.892711,-0.146041,-0.740313,-0.403244,-0.615800,-0.039733,1,False,False,True,15899_163621


In [260]:
data_filtered["is_anomaly"] = (anomaly_predictions == -1).astype(int)

merged_data = pd.merge(
    data,
    data_filtered[["house_tkn", "flat_tkn", "payment_period", "is_anomaly"]],
    on=["house_tkn", "flat_tkn", "payment_period"],
    how="left",
)

data["is_anomaly"] = merged_data["is_anomaly"]

In [262]:
data.to_csv("original_data_with_anomaly_column.csv", sep=";", index=False)

## Correlation

In [ ]:
plt.figure(figsize=(10, 6))
number_features = ['income', 'debt', 'raised','volume_cold', 'volume_hot', 'volume_electr']
data_with_real_numeric = data[number_features].dropna() 
data_with_real_numeric = (data_with_real_numeric-data_with_real_numeric.mean())/data_with_real_numeric.std()
pearson = data_with_real_numeric.corr(method = 'pearson')
sns.heatmap(pearson, annot = True, fmt = ".3f")
plt.title('Pearson correlation of numerical data')

In [ ]:
plt.figure(figsize=(10, 6))
number_features = ['income', 'debt', 'raised','volume_cold', 'volume_hot', 'volume_electr']
data_with_real_numeric = data[number_features].dropna() 
data_with_real_numeric = (data_with_real_numeric-data_with_real_numeric.mean())/data_with_real_numeric.std()
spearman = data_with_real_numeric.corr(method = 'spearman')
sns.heatmap(spearman, annot = True, fmt = ".3f")
plt.title('Spearman correlation of numerical data')

In [ ]:
plt.figure(figsize=(10, 6))
number_features = ['income', 'debt', 'raised','volume_cold', 'volume_hot', 'volume_electr']
data_with_real_numeric = data[number_features]
data_with_real_numeric = (data_with_real_numeric-data_with_real_numeric.mean())/data_with_real_numeric.std()
spearman = data_with_real_numeric.corr(method = 'spearman')
sns.heatmap(spearman, annot = True, fmt = ".3f")
plt.title('Spearman correlation of numerical data including NaNs')

### Correlation on normal data

In [ ]:
os.listdir("data")

In [ ]:
data_filtered_normal_scaled = pd.read_csv("data//filtered_scaled_data.csv", sep=";")

In [ ]:
plt.figure(figsize=(10, 6))
number_features = ['income', 'debt', 'raised','volume_cold', 'volume_hot', 'volume_electr']
data_with_real_numeric = data_filtered_normal_scaled.dropna() 
data_with_real_numeric = (data_with_real_numeric-data_with_real_numeric.mean())/data_with_real_numeric.std()
pearson = data_with_real_numeric.corr(method = 'pearson')
sns.heatmap(pearson, annot = True, fmt = ".3f")
plt.title('Pearson correlation of numerical data')

In [ ]:
plt.figure(figsize=(10, 6))
number_features = ['income', 'debt', 'raised','volume_cold', 'volume_hot', 'volume_electr']
data_with_real_numeric = data_filtered_normal_scaled.dropna() 
data_with_real_numeric = (data_with_real_numeric-data_with_real_numeric.mean())/data_with_real_numeric.std()
spearman = data_with_real_numeric.corr(method = 'spearman')
sns.heatmap(spearman, annot = True, fmt = ".3f")
plt.title('Spearman correlation of numerical data')

# Кластеризация данных

- если делать деревом решений то не должно возникать проблемы с NaN значениями
- если делать стандартными методам сначала надо нормализовать и разделить данные на NaN и все остальные, либо выделять тех у кого только холодная вода с целью заставить им эту воду 0, а остальных кокнуть как аномалии вручную. Некоторые стандартные методы: K-Means, DBSCAN

**До начала кластеризации** надо убрать дома и квартиры объединив их в $user\_id$. Дальше надо подумать о том каие признаки можно дабвить в табличку (например максимум, минимум, среднее, отклонение, частота выплат). Также можно пытаться учесть отдельно
1. летний период отключения горячей воды
2. отсутствие горячей воды в доме
3. периоды повышенного потребления воды, например столбца по типу +1 за зимний месяц, -1 за летний или вообще булевые (для K-means очевидно плохая идея)

In [ ]:
data['user_id'] = data['house_tkn'].astype(str) + '_' + data['flat_tkn'].astype(str)

grouped = data.groupby('user_id')
user_features = grouped.agg({
    'income': 'mean',
    'debt': 'mean',
    'raised': 'mean',
    'volume_cold': ['mean', 'std'],
    'volume_hot': ['mean', 'std'],
    'volume_electr': ['mean', 'std'],
    'payment_period': ['min', 'max', 'nunique'],
})

user_features.columns = ['_'.join(col) for col in user_features.columns]

# 3. Extract Payment Pattern Features

# Create a column for payment delay
user_features['payment_delay'] = (user_features['payment_period_max'] - user_features['payment_period_min']).dt.days
user_features['payment_delay'] /= user_features['payment_period_nunique']  # Average delay per month